In [167]:
import ast

import pandas as pd

import datetime

from keras.layers import Input, Dense, Embedding, merge, Flatten, Merge, BatchNormalization
from keras.models import Model, load_model
from keras.regularizers import l2
import keras.backend as K
from keras.optimizers import SGD
import numpy as np

from sklearn.cluster import MeanShift, estimate_bandwidth

import utils

import data

from sklearn.model_selection import train_test_split

from bcolz_array_iterator import BcolzArrayIterator

import bcolz

from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import ModelCheckpoint

import pickle

Using Theano backend.


AttributeError: module 'theano' has no attribute 'tests'

Data Path

In [59]:
data_path = "/data/datasets/taxi/"

#  CSV 2 DATA

Need to check process and see if resulting data is the same

## Original data

This is the author's feature extraction process. I have modified it so that it returns the tuple instead of saving as hdf5 file

In [60]:
import ast
import csv
import os
import sys

import h5py
import numpy
from fuel.converters.base import fill_hdf5_file

import data


taxi_id_dict = {}
origin_call_dict = {0: 0}

def get_unique_taxi_id(val):
    if val in taxi_id_dict:
        return taxi_id_dict[val]
    else:
        taxi_id_dict[val] = len(taxi_id_dict)
        return len(taxi_id_dict) - 1

def get_unique_origin_call(val):
    if val in origin_call_dict:
        return origin_call_dict[val]
    else:
        origin_call_dict[val] = len(origin_call_dict)
        return len(origin_call_dict) - 1

def read_stands(input_directory, h5file):
    stands_name = numpy.empty(shape=(data.stands_size,), dtype=('a', 24))
    stands_latitude = numpy.empty(shape=(data.stands_size,), dtype=numpy.float32)
    stands_longitude = numpy.empty(shape=(data.stands_size,), dtype=numpy.float32)
    stands_name[0] = 'None'
    stands_latitude[0] = stands_longitude[0] = 0
    with open(os.path.join(input_directory, 'metaData_taxistandsID_name_GPSlocation.csv'), 'r') as f:
        reader = csv.reader(f)
        next(reader) # header
        for line in reader:
            id = int(line[0])
            stands_name[id] = line[1].encode('utf-8')
            stands_latitude[id] = float(line[2])
            stands_longitude[id] = float(line[3])
    return (('stands', 'stands_name', stands_name),
            ('stands', 'stands_latitude', stands_latitude),
            ('stands', 'stands_longitude', stands_longitude))

def read_taxis(input_directory, h5file, dataset):
    size=getattr(data, '%s_size'%dataset)
    trip_id = numpy.empty(shape=(size,), dtype='S19')
    call_type = numpy.empty(shape=(size,), dtype=numpy.int8)
    origin_call = numpy.empty(shape=(size,), dtype=numpy.int32)
    origin_stand = numpy.empty(shape=(size,), dtype=numpy.int8)
    taxi_id = numpy.empty(shape=(size,), dtype=numpy.int16)
    timestamp = numpy.empty(shape=(size,), dtype=numpy.int32)
    day_type = numpy.empty(shape=(size,), dtype=numpy.int8)
    missing_data = numpy.empty(shape=(size,), dtype=numpy.bool)
    latitude = numpy.empty(shape=(size,), dtype=data.Polyline)
    longitude = numpy.empty(shape=(size,), dtype=data.Polyline)
    with open(os.path.join(input_directory, '%s.csv'%dataset), 'r') as f:
        reader = csv.reader(f)
        next(reader) # header
        id=0
        for line in reader:
            trip_id[id] = line[0].encode('utf-8')
            call_type[id] = ord(line[1][0]) - ord('A')
            origin_call[id] = 0 if line[2]=='NA' or line[2]=='' else get_unique_origin_call(int(line[2]))
            origin_stand[id] = 0 if line[3]=='NA' or line[3]=='' else int(line[3])
            taxi_id[id] = get_unique_taxi_id(int(line[4]))
            timestamp[id] = int(line[5])
            day_type[id] = ord(line[6][0]) - ord('A')
            missing_data[id] = line[7][0] == 'T'
            polyline = ast.literal_eval(line[8])
            latitude[id] = numpy.array([point[1] for point in polyline], dtype=numpy.float32)
            longitude[id] = numpy.array([point[0] for point in polyline], dtype=numpy.float32)
            id+=1
    splits = ()
    for name in ['trip_id', 'call_type', 'origin_call', 'origin_stand', 'taxi_id', 'timestamp', 'day_type', 'missing_data', 'latitude', 'longitude']:
        splits += ((dataset, name, locals()[name]),)
    return splits

def unique(h5file):
    unique_taxi_id = numpy.empty(shape=(data.taxi_id_size,), dtype=numpy.int32)
    assert len(taxi_id_dict) == data.taxi_id_size
    for k, v in taxi_id_dict.items():
        unique_taxi_id[v] = k

    unique_origin_call = numpy.empty(shape=(data.origin_call_size,), dtype=numpy.int32)
    assert len(origin_call_dict) == data.origin_call_size
    for k, v in origin_call_dict.items():
        unique_origin_call[v] = k

    return (('unique_taxi_id', 'unique_taxi_id', unique_taxi_id),
            ('unique_origin_call', 'unique_origin_call', unique_origin_call))

def get_data(input_directory):
    split = ()
    split += read_stands(input_directory, None)
    split += read_taxis(input_directory, None, 'train')
    split += read_taxis(input_directory, None, 'test')
    split += unique(None)
    return split


### Check

manually go through data collection

In [61]:
taxi_id_dict = {}
origin_call_dict = {0: 0}

In [62]:
split = ()

In [63]:
split += read_stands(data_path+'data', None)


In [ ]:
split += read_taxis(data_path+'data', None, 'train')


In [ ]:
split += read_taxis(data_path+'data', None, 'test')

In [ ]:
split += unique(None)


Data structure: Tuple of tuples. Each sub-tuple: ('dataset', 'column' 'data')

Contains stands: metadata. 

Save tulpe

In [80]:
with open(data_path+'/data/data_tuple.pickle', 'wb') as f:
    pickle.dump(split, f)

Load tuple

In [ ]:
with open(data_path+'/data/data_tuple.pickle', 'r') as f:
     split = pickle.load(f)

### Validation Split

Time cuts

In [168]:
cuts = [
    1376503200, # 2013-08-14 18:00
    1380616200, # 2013-10-01 08:30
    1381167900, # 2013-10-07 17:45
    1383364800, # 2013-11-02 04:00
    1387722600  # 2013-12-22 14:30
]

In [191]:
split[12][2][0]

array([-8.61864281, -8.6184988 , -8.62032604, -8.62215328, -8.62395287,
       -8.62668037, -8.62737274, -8.63022614, -8.63274574, -8.63173771,
       -8.62993813, -8.62911034, -8.62912846, -8.62878609, -8.6286869 ,
       -8.62875938, -8.63083839, -8.63232327, -8.63114357, -8.63082886,
       -8.63082886, -8.63082886, -8.63083839], dtype=float32)

In [192]:
split

(('stands',
  'stands_name',
  array([b'None', b'Agra', b'Alameda', b'Aldoar', b'Alf\xc3\xa2ndega',
         b'Amial', b'Areosa', b'Av. Boavista', b'Azevedo', b'Batalha',
         b'Bolh\xc3\xa3o', b'Bom Pastor', b'Bom Sucesso', b'Bras\xc3\xadlia',
         b'C\xc3\xa2mara', b'Campanh\xc3\xa3', b'Campismo', b'Carcereira',
         b'Carregal', b'Carvalheiras', b'Carvalhido', b'Casa da Musica',
         b'Castelo do Queijo', b'Cl\xc3\xa9rigos', b'Conde Ferreira',
         b'Cordoaria', b'Corujeira', b'D. Jo\xc3\xa3o I', b'Drag\xc3\xa3o',
         b'Francos', b'Galiza', b'H. Magalh\xc3\xa3es Lemos', b'H. Militar',
         b'H. S\xc3\xa3o Jo\xc3\xa3o', b'Infante', b'Lago', b'Lordelo',
         b'Marechal', b'Marqu\xc3\xaas', b'Maternidade', b'Mercado da Foz',
         b'Nevogilde', b'Nove de Abril', b'Ouro', b'Pal\xc3\xa1cio',
         b'Passeio Alegre', b'Pereira de Melo', b'Pereir\xc3\xb3',
         b'Pinto de Azevedo', b'P\xc3\xb3lo Universit\xc3\xa1rio',
         b'Prelada', b'Ramada

In [203]:
split[7][1]

'taxi_id'

In [224]:
def make_valid(split):

    valid = (
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        []
    )

    for i in range(len(split[5][2])):
        trip_id = split[3][2][i]
        call_type = split[4][2][i]
        origin_call = split[5][2][i]
        origin_stand = split[6][2][i]
        taxi_id = split[7][2][i]
        time = split[8][2][i]
        day_type = split[9][2][i]
        missing_data = split[10][2][i]
        latitude = split[11][2][i]
        longitude = split[12][2][i]

        if len(latitude) == 0:
            continue

        for ts in cuts:
            if time <= ts and time + 15 * (len(latitude) - 1) >= ts:
                # keep it
                valid[0].append(trip_id)
                valid[1].append(call_type)
                valid[2].append(origin_call)
                valid[3].append(origin_stand)
                valid[4].append(taxi_id)
                valid[5].append(time)
                valid[6].append(day_type)
                valid[7].append(missing_data)
                n = (ts - time) / 15 + 1
                valid[8].append(latitude[:n])
                valid[9].append(longitude[:n])
                valid[10].append(latitude[-1])
                valid[11].append(longitude[-1])
                valid[12].append(15 * (len(latitude)-1))
                break
    return (
        ('valid', 'trip_id', np.array(valid[0])),
        ('valid', 'call_type', np.array(valid[1])),
        ('valid', 'origin_call', np.array(valid[2])),
        ('valid', 'origin_stand', np.array(valid[3])),
        ('valid', 'taxi_id', np.array(valid[4])),
        ('valid', 'timestamp', np.array(valid[5])),
        ('valid', 'day_type', np.array(valid[6])),
        ('valid', 'missing_data', np.array(valid[7])),
        ('valid', 'latitude', np.array(valid[8])),
        ('valid', 'longitude', np.array(valid[9])),
        ('valid', 'destination_latitude', np.array(valid[10])),
        ('valid', 'destination_longitude', np.array(valid[11])),
        ('valid', 'travel_time', np.array(valid[12]))
    )

In [225]:
valid_split = make_valid(split)

/home/bckenstler/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:46: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/bckenstler/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:47: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [226]:
valid_split

(('valid', 'trip_id', array([b'1376502576620000126', b'1376503146620000161',
         b'1376502942620000500', b'1376502604620000105',
         b'1376502611620000022', b'1376503017620000272',
         b'1376501811620000617', b'1376502255620000663',
         b'1376502950620000005', b'1376502635620000276',
         b'1376502627620000596', b'1376502237620000675',
         b'1376500608620000409', b'1376502047620000574',
         b'1376502355620000338', b'1376502180620000080',
         b'1376502802620000680', b'1376502603620000142',
         b'1376501182620000651', b'1376503155620000026',
         b'1376502617620000657', b'1376502994620000604',
         b'1376502822620000093', b'1376502466620000561',
         b'1376503126620000410', b'1376501063620000343',
         b'1376502919620000166', b'1376503023620000010',
         b'1376502971620000517', b'1376503012620000273',
         b'1376503077620000470', b'1376500095620000569',
         b'1376502674620000426', b'1376503111620000674',
         b'

## Pandas Data

### Meta-data

In [95]:
stands =  pd.read_csv(data_path+'/data/metaData_taxistandsID_name_GPSlocation.csv', header=0)

In [96]:
stands.head()

,ID,Descricao,Latitude,Longitude
0,1,Agra,41.177146,-8.609670
1,2,Alameda,41.156190,-8.591064
2,3,Aldoar,41.170525,-8.665876
3,4,Alfândega,41.143764,-8.621803
4,5,Amial,41.183510,-8.612726


Compare columns

In [97]:
stands['Descricao']

0                         Agra
1                      Alameda
2                       Aldoar
3                    Alfândega
4                        Amial
5                       Areosa
6                 Av. Boavista
7                      Azevedo
8                      Batalha
9                       Bolhão
10                  Bom Pastor
11                 Bom Sucesso
12                    Brasília
13                      Câmara
14                    Campanhã
15                    Campismo
16                  Carcereira
17                    Carregal
18                Carvalheiras
19                  Carvalhido
20              Casa da Musica
21           Castelo do Queijo
22                    Clérigos
23              Conde Ferreira
24                   Cordoaria
25                   Corujeira
26                   D. João I
27                      Dragão
28                     Francos
29                      Galiza
                ...           
33                     Infante
34      

In [98]:
len(stands['Descricao'].as_matrix())

63

In [99]:
split[2][2]

array([ 0.        , -8.60967064, -8.59106445, -8.66587639, -8.62180328,
       -8.61272621, -8.58278275, -8.64198399, -8.56819534, -8.60571957,
       -8.60690975, -8.61326694, -8.63046646, -8.62846756, -8.61107063,
       -8.58587646, -8.6277914 , -8.63228607, -8.61960316, -8.60542679,
       -8.6267786 , -8.62956524, -8.68918037, -8.61257458, -8.58934784,
       -8.61763763, -8.57977581, -8.60924149, -8.58404636, -8.6386652 ,
       -8.62775898, -8.66396046, -8.62704372, -8.60012245, -8.61595249,
       -8.6495266 , -8.6490736 , -8.657547  , -8.60424614, -8.62279987,
       -8.67400646, -8.67598343, -8.61178398, -8.64888668, -8.62578487,
       -8.6727953 , -8.64853573, -8.65449619, -8.64505005, -8.60253906,
       -8.62890244, -8.62112236, -8.61321735, -8.61401272, -8.63037491,
       -8.5703516 , -8.59164524, -8.61070728, -8.60452843, -8.64120483,
       -8.60984325, -8.60030842, -8.64655781, -8.60936928], dtype=float32)

In [100]:
len(split[0][2])

64

Author's data has a "None" row w/ zeros. Will add to series

In [101]:
d={'col1': 1, 'col2': 2}

In [102]:
stands = pd.DataFrame([['None',0.,0.]], columns=['Descricao', 'Latitude', 'Longitude']).append(stands)

In [103]:
np.allclose(stands['Latitude'],split[1][2])

True

In [104]:
np.allclose(stands['Longitude'],split[2][2])

True

Longs/Lats check out

### Train Data

In [105]:
data = pd.read_csv(data_path+'data/train.csv', header=0)

In [169]:
data.columns

Index(['TRIP_ID', 'CALL_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID',
       'TIMESTAMP', 'DAY_TYPE', 'MISSING_DATA', 'POLYLINE'],
      dtype='object')

Check data same size

In [106]:
data.shape

(1710670, 9)

In [107]:
len(split[4][2])

1710670

In [245]:
split[4][1]

'call_type'

#### CAll type

In [242]:
call_type_f = lambda x:ord(x) - ord('A')

In [247]:
np.allclose(data['CALL_TYPE'].apply(call_type_f),split[4][2])

True

In [248]:
data['CALL_TYPE'] = data['CALL_TYPE'].apply(call_type_f)

#### Origin Call

Turn origin call into categorical variable

Can do using factorize: we want the nulls to be zero. Factorize sets them as -1, add 1 to set as needed

In [110]:
 np.allclose(pd.Series(pd.factorize(data['ORIGIN_CALL'])[0])+1,split[5][2])

True

And it is the same

In [111]:
data['ORIGIN_CALL'] = pd.Series(pd.factorize(data['ORIGIN_CALL'])[0]) + 1

In [113]:
num_origin_call = len(data['ORIGIN_CALL'].unique())

In [79]:
s = set()

In [ ]:
s.i

#### Origin Stand

In [283]:
origin_stand_f = lambda x: 0 if pd.isnull(x) or x=='' else int(x)

In [178]:
split[6]

('train', 'origin_stand', array([ 0,  7,  0, ...,  0, 12, 34], dtype=int8))

In [284]:
np.allclose(data["ORIGIN_STAND"].apply(origin_stand_f),split[6][2])

True

In [285]:
data["ORIGIN_STAND"] = data["ORIGIN_STAND"].apply(origin_stand_f)

In [114]:
num_origin_stand = len(data['ORIGIN_STAND'].unique())

In [115]:
num_origin_stand

64

#### Taxi ID

In [116]:
split[7][2]

array([  0,   1,   2, ..., 439, 443, 444], dtype=int16)

In [117]:
pd.factorize(data['TAXI_ID'])[0]

array([  0,   1,   2, ..., 439, 443, 444])

In [118]:
np.allclose(pd.Series(pd.factorize(data['TAXI_ID'])[0]),split[7][2])

True

In [119]:
data['TAXI_ID'] = pd.Series(pd.factorize(data['TAXI_ID'])[0])

In [120]:
num_taxi_id = data['TAXI_ID'].unique()

#### Day Type

In [121]:
split[9]

('train', 'day_type', array([0, 0, 0, ..., 0, 0, 0], dtype=int8))

In [122]:
day_type_f = lambda x: ord(x[0]) - ord('A')

In [123]:
np.allclose(data['DAY_TYPE'].apply(day_type_f),split[9][2])

True

In [124]:
data['DAY_TYPE'] = data['DAY_TYPE'].apply(day_type_f)

#### Long/lat

In [203]:
polyline_f = lambda x: ast.literal_eval(x)

In [204]:
polyline = data['POLYLINE'].apply(polyline_f)

In [205]:
polyline.to_pickle(data_path+'data/polylines.pkl')

In [125]:
polyline = pd.read_pickle(data_path+'data/polylines.pkl')

In [126]:
len(polyline)

1710670

In [208]:
polyline

0          [[-8.618643, 41.141412], [-8.618499, 41.141376...
1          [[-8.639847, 41.159826], [-8.640351, 41.159871...
2          [[-8.612964, 41.140359], [-8.613378, 41.14035]...
3          [[-8.574678, 41.151951], [-8.574705, 41.151942...
4          [[-8.645994, 41.18049], [-8.645949, 41.180517]...
5          [[-8.615502, 41.140674], [-8.614854, 41.140926...
6          [[-8.57952, 41.145948], [-8.580942, 41.145039]...
7          [[-8.617563, 41.146182], [-8.617527, 41.145849...
8          [[-8.611794, 41.140557], [-8.611785, 41.140575...
9          [[-8.615907, 41.140557], [-8.614449, 41.141088...
10         [[-8.619894, 41.148009], [-8.620164, 41.14773]...
11         [[-8.56242, 41.168403], [-8.562429, 41.168358]...
12         [[-8.618868, 41.155101], [-8.6175, 41.154912],...
13         [[-8.608716, 41.153499], [-8.607627, 41.153481...
14         [[-8.612208, 41.14053], [-8.612235, 41.140521]...
15         [[-8.585145, 41.164857], [-8.584146, 41.164704...
16         [[-8.584263, 

In [127]:
lats =  pd.Series([np.array([point[1] for point in poly],dtype=np.float32) for poly in polyline])

In [233]:
split[11][2]

array([ array([ 41.1414,  41.1414,  41.1425,  41.1438,  41.1444,  41.1448,  41.1447,  41.1452,  41.1469,
        41.1482,  41.1504,  41.1512,  41.1512,  41.1522,  41.1524,  41.1525,  41.1527,  41.153 ,
        41.1545,  41.1545,  41.1545,  41.1545,  41.1545], dtype=float32),
       array([ 41.1598,  41.1599,  41.1601,  41.1605,  41.1609,  41.1615,  41.162 ,  41.1626,  41.1632,
        41.1637,  41.1642,  41.1647,  41.1651,  41.1666,  41.168 ,  41.169 ,  41.17  ,  41.1706,
        41.1707], dtype=float32),
       array([ 41.1404,  41.1404,  41.1403,  41.1404,  41.1404,  41.1406,  41.1414,  41.1428,  41.1441,
        41.1448,  41.1447,  41.1452,  41.1465,  41.1473,  41.1488,  41.1506,  41.1521,  41.1525,
        41.1527,  41.1528,  41.1528,  41.1526,  41.153 ,  41.1536,  41.154 ,  41.1545,  41.1543,
        41.1543,  41.1543,  41.1544,  41.1543,  41.1543,  41.1543,  41.1543,  41.1417,  41.1406,
        41.1543,  41.1538,  41.1538,  41.1538,  41.1532,  41.1526,  41.1524,  41.1512,  41.150

In [128]:
np.alltrue([np.allclose(lats[i],split[11][2][i]) for i in range(len(lats))])

True

Latitudes check out

In [129]:
longs =  pd.Series([np.array([point[0] for point in poly],dtype=np.float32) for poly in polyline])

In [241]:
split[12]

('train',
 'longitude',
 array([ array([-8.6186, -8.6185, -8.6203, -8.6222, -8.624 , -8.6267, -8.6274, -8.6302, -8.6327, -8.6317,
        -8.6299, -8.6291, -8.6291, -8.6288, -8.6287, -8.6288, -8.6308, -8.6323, -8.6311, -8.6308,
        -8.6308, -8.6308, -8.6308], dtype=float32),
        array([-8.6398, -8.6404, -8.6422, -8.6445, -8.6469, -8.65  , -8.6532, -8.6564, -8.6602, -8.6631,
        -8.6662, -8.6692, -8.6709, -8.6709, -8.6696, -8.6681, -8.6666, -8.6658, -8.6657], dtype=float32),
        array([-8.613 , -8.6134, -8.6142, -8.6148, -8.6159, -8.6166, -8.6185, -8.6206, -8.6226, -8.6251,
        -8.6274, -8.6301, -8.6319, -8.6326, -8.6313, -8.6297, -8.6288, -8.6286, -8.6287, -8.6304,
        -8.6327, -8.6351, -8.6377, -8.6404, -8.6422, -8.6441, -8.6465, -8.6486, -8.6495, -8.6498,
        -8.6498, -8.6499, -8.6499, -8.6499, -8.5994, -8.5965, -8.6501, -8.6504, -8.6504, -8.6504,
        -8.6499, -8.6494, -8.6492, -8.6497, -8.6491, -8.6491, -8.6486, -8.6484, -8.6476, -8.6459,
        -8.6

In [130]:
np.alltrue([np.allclose(longs[i],split[12][2][i]) for i in range(len(longs))])

True

Longitudes check out

In [131]:
data['LATITUDE'] = lats

In [132]:
data['LONGITUDE'] = longs

In [134]:
data

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,LATITUDE,LONGITUDE
0,1372636858620000589,C,0,NaN,0,1372636858,0,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...","[41.1414, 41.1414, 41.1425, 41.1438, 41.1444, ...","[-8.61864, -8.6185, -8.62033, -8.62215, -8.623..."
1,1372637303620000596,B,0,7.0,1,1372637303,0,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[...","[41.1598, 41.1599, 41.1601, 41.1605, 41.1609, ...","[-8.63985, -8.64035, -8.6422, -8.64445, -8.646..."
2,1372636951620000320,C,0,NaN,2,1372636951,0,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...","[41.1404, 41.1404, 41.1403, 41.1404, 41.1404, ...","[-8.61296, -8.61338, -8.61421, -8.61477, -8.61..."
3,1372636854620000520,C,0,NaN,3,1372636854,0,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[...","[41.152, 41.1519, 41.1519, 41.152, 41.1519, 41...","[-8.57468, -8.57471, -8.5747, -8.57466, -8.574..."
4,1372637091620000337,C,0,NaN,4,1372637091,0,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...","[41.1805, 41.1805, 41.18, 41.1789, 41.1785, 41...","[-8.64599, -8.64595, -8.64605, -8.6468, -8.649..."
5,1372636965620000231,C,0,NaN,5,1372636965,0,False,"[[-8.615502,41.140674],[-8.614854,41.140926],[...","[41.1407, 41.1409, 41.1415, 41.1409, 41.1413, ...","[-8.6155, -8.61485, -8.61335, -8.60998, -8.607..."
6,1372637210620000456,C,0,NaN,6,1372637210,0,False,"[[-8.57952,41.145948],[-8.580942,41.145039],[-...","[41.1459, 41.145, 41.145, 41.1462, 41.1468, 41...","[-8.57952, -8.58094, -8.58271, -8.58409, -8.58..."
7,1372637299620000011,C,0,NaN,7,1372637299,0,False,"[[-8.617563,41.146182],[-8.617527,41.145849],[...","[41.1462, 41.1458, 41.1448, 41.1454, 41.1454, ...","[-8.61756, -8.61753, -8.61698, -8.61575, -8.61..."
8,1372637274620000403,C,0,NaN,8,1372637274,0,False,"[[-8.611794,41.140557],[-8.611785,41.140575],[...","[41.1406, 41.1406, 41.1406, 41.1405, 41.1403, ...","[-8.61179, -8.61178, -8.612, -8.61262, -8.6137..."
9,1372637905620000320,C,0,NaN,2,1372637905,0,False,"[[-8.615907,41.140557],[-8.614449,41.141088],[...","[41.1406, 41.1411, 41.1414, 41.1408, 41.1395, ...","[-8.61591, -8.61445, -8.61352, -8.6099, -8.609..."


SAVE DICTS

In [308]:
np.save(data_path+'data/origin_call_dict.npy', origin_call_dict)

In [309]:
np.save(data_path+'data/taxi_id_dict.npy', taxi_id_dict)

### Test DATA

In [135]:
test_data = pd.read_csv(data_path+'data/test.csv', header=0)

In [136]:
test_data.columns

Index(['TRIP_ID', 'CALL_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID',
       'TIMESTAMP', 'DAY_TYPE', 'MISSING_DATA', 'POLYLINE'],
      dtype='object')

Check data same size

In [137]:
test_data.shape

(320, 9)

In [138]:
len(split[13][2])

320

#### CAll type

In [242]:
call_type_f = lambda x:ord(x) - ord('A')

In [257]:
np.allclose(test_data['CALL_TYPE'].apply(call_type_f),split[14][2])

True

In [258]:
test_data['CALL_TYPE'] = test_data['CALL_TYPE'].apply(call_type_f)

#### Origin Call

Turn origin call into categorical variable

Can do using factorize: we want the nulls to be zero. Factorize sets them as -1, add 1 to set as needed

In [75]:
np.unique(split[15][2])

array([    0,     7,   249,   502,   623,   935,  1729,  2939,  3476,
        3520,  3613,  5145,  7017,  7178,  8316,  8326,  8701,  9843,
       10833, 11099, 11134, 16141, 16613, 16658, 20375, 21655, 22338,
       23356, 23614, 23831, 33467, 34117, 34224, 34725, 35492, 35708,
       36140, 36905, 41178, 42039, 43989, 47093, 57106, 57107, 57108,
       57109, 57110, 57111, 57112, 57113, 57114, 57115, 57116, 57117,
       57118, 57119, 57120, 57121, 57122, 57123, 57124], dtype=int32)

Hold up! We need to use our previous mapping

In [9]:
import numpy as np

In [47]:
taxi_id_dict = np.load(data_path+'data/taxi_id_dict.npy').item()

In [139]:
len(origin_call_dict)

57125

In [140]:
test_origin_call_f = lambda x: 0 if (np.isnan(x) or x=='' or x >= num_origin_call) else origin_call_dict[x]

In [141]:
np.allclose(test_data['ORIGIN_CALL'].apply(test_origin_call_f),split[15][2])

False

In [142]:
test_data['ORIGIN_CALL'] = test_data['ORIGIN_CALL'].apply(test_origin_call_f)

#### Origin Stand

In [144]:
origin_stand_f = lambda x: 0 if pd.isnull(x) or x=='' else int(x)

In [269]:
split[16]

('test',
 'origin_stand',
 array([15, 57, 15, 53, 18,  0, 15,  0,  9, 15, 56,  0,  0,  0,  0,  0,  0,  0, 15,  0, 15,  0,  0,
        15, 12, 49, 15, 15, 21, 15, 15, 57,  0,  0,  0,  0,  0, 60, 15, 34, 38,  0,  0,  9, 30, 61,
         0,  0, 28, 56, 13, 10, 60, 15, 15, 14,  9, 35,  0,  0, 15,  0,  0,  0,  9,  9, 15,  0,  0,
        13,  0,  0, 15,  0, 47,  0, 23,  0,  0,  0,  0, 59,  0,  0,  7,  0,  0,  0, 16,  0,  0,  0,
         0,  0, 60,  0,  0,  0,  0,  0,  7, 33,  0, 57, 57,  0, 25, 11, 33,  0,  0,  0, 42,  0,  0,
         0, 35,  6,  0, 36,  0,  0,  0,  0, 57,  0,  0,  0,  0, 57, 57, 47,  0,  0,  0,  0, 15, 35,
         0,  0,  0, 57,  0, 15,  0,  0, 16,  0, 33,  3,  0,  0,  0,  0,  0, 17, 25,  0, 57, 15,  0,
         0,  0, 35,  0, 60,  0,  0,  0, 18, 40, 49,  0, 13, 38,  0,  9,  0,  0,  0, 36, 53,  0,  0,
        34,  0,  0, 14, 53,  0,  0, 28,  0,  0, 25,  0,  0, 28,  0, 36,  0,  0, 47,  0,  0, 26, 18,
         0,  0,  0, 57, 53,  0, 10, 54,  0,  6, 63, 21, 15,  0,  0,  0,  0

In [145]:
np.allclose(test_data["ORIGIN_STAND"].apply(origin_stand_f),split[16][2])

True

In [146]:
test_data["ORIGIN_STAND"] = test_data["ORIGIN_STAND"].apply(origin_stand_f)

#### Taxi ID

In [147]:
split[17][2]

array([150, 306, 260,  40,  33, 174, 335, 315,  32, 364, 199, 393, 334,
       423,  11, 232, 269, 285,  69, 371, 190,  52,  92, 402, 359, 112,
       406, 427, 198, 248, 157, 209, 308, 264, 146, 125, 283, 266, 360,
       104, 241, 263,  56, 224,  21,  80,  98, 318, 254, 303, 249, 227,
       367,  96,  19, 193, 357, 361, 339, 375, 362, 270, 267, 129, 351,
       323, 310, 416, 181, 281,  25,  43, 385, 117, 211, 235,  25, 399,
       122, 341, 215, 198, 165,  81, 108, 221,  89, 380, 392, 231,  35,
        39,  26, 138,  79, 386,  30, 146,  41, 105, 333, 126,  37, 120,
       166, 255, 297, 116, 206, 186, 404, 242, 325, 246, 154, 175, 413,
       284, 288,   1, 443, 409, 416,  13,  27, 224, 176, 345, 118,   0,
        91, 371,  32,  57, 326, 254, 283, 287, 271, 319, 115, 179, 328,
       424,  49, 210, 238, 232, 298, 412, 334, 440, 151,  16, 156, 231,
        49, 287, 428, 342, 169, 267,  87, 289, 305, 307, 380, 292, 341,
       205, 276, 355,   8,  20, 324, 391, 402, 144, 225, 337,  8

In [148]:
test_taxi_id_f = lambda x: taxi_id_dict[x]

In [149]:
np.allclose(test_data['TAXI_ID'].apply(test_taxi_id_f),split[17][2])

True

In [150]:
test_data['TAXI_ID'] = test_data['TAXI_ID'].apply(test_taxi_id_f)

In [152]:
test_data['ORIGIN_CALL'].unique()

array([    0, 10833, 35708,   623,     7,  7017, 23356, 16658,  7178,
       34224, 41178,   935,  3520, 47093,   249, 34725, 11099, 35492,
        3613, 23831, 22338,  5145, 34117,   502, 33467, 21655, 23614,
       16613,  1729,  8326, 20375,  8316, 16141,  3476, 11134, 42039,
       36905,  2939, 36140])

In [153]:
test_data['TAXI_ID'].as_matrix()

array([150, 306, 260,  40,  33, 174, 335, 315,  32, 364, 199, 393, 334,
       423,  11, 232, 269, 285,  69, 371, 190,  52,  92, 402, 359, 112,
       406, 427, 198, 248, 157, 209, 308, 264, 146, 125, 283, 266, 360,
       104, 241, 263,  56, 224,  21,  80,  98, 318, 254, 303, 249, 227,
       367,  96,  19, 193, 357, 361, 339, 375, 362, 270, 267, 129, 351,
       323, 310, 416, 181, 281,  25,  43, 385, 117, 211, 235,  25, 399,
       122, 341, 215, 198, 165,  81, 108, 221,  89, 380, 392, 231,  35,
        39,  26, 138,  79, 386,  30, 146,  41, 105, 333, 126,  37, 120,
       166, 255, 297, 116, 206, 186, 404, 242, 325, 246, 154, 175, 413,
       284, 288,   1, 443, 409, 416,  13,  27, 224, 176, 345, 118,   0,
        91, 371,  32,  57, 326, 254, 283, 287, 271, 319, 115, 179, 328,
       424,  49, 210, 238, 232, 298, 412, 334, 440, 151,  16, 156, 231,
        49, 287, 428, 342, 169, 267,  87, 289, 305, 307, 380, 292, 341,
       205, 276, 355,   8,  20, 324, 391, 402, 144, 225, 337,  8

#### Day Type

In [154]:
split[19]

('test',
 'day_type',
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [155]:
day_type_f = lambda x: ord(x[0]) - ord('A')

In [156]:
np.allclose(test_data['DAY_TYPE'].apply(day_type_f),split[19][2])

True

In [157]:
test_data['DAY_TYPE'] = test_data['DAY_TYPE'].apply(day_type_f)

#### Long/lat

In [158]:
polyline_f = lambda x: ast.literal_eval(x)

In [159]:
test_polyline = test_data['POLYLINE'].apply(polyline_f)

In [160]:
test_polyline.to_pickle(data_path+'data/test_polylines.pkl')

In [208]:
polyline

0          [[-8.618643, 41.141412], [-8.618499, 41.141376...
1          [[-8.639847, 41.159826], [-8.640351, 41.159871...
2          [[-8.612964, 41.140359], [-8.613378, 41.14035]...
3          [[-8.574678, 41.151951], [-8.574705, 41.151942...
4          [[-8.645994, 41.18049], [-8.645949, 41.180517]...
5          [[-8.615502, 41.140674], [-8.614854, 41.140926...
6          [[-8.57952, 41.145948], [-8.580942, 41.145039]...
7          [[-8.617563, 41.146182], [-8.617527, 41.145849...
8          [[-8.611794, 41.140557], [-8.611785, 41.140575...
9          [[-8.615907, 41.140557], [-8.614449, 41.141088...
10         [[-8.619894, 41.148009], [-8.620164, 41.14773]...
11         [[-8.56242, 41.168403], [-8.562429, 41.168358]...
12         [[-8.618868, 41.155101], [-8.6175, 41.154912],...
13         [[-8.608716, 41.153499], [-8.607627, 41.153481...
14         [[-8.612208, 41.14053], [-8.612235, 41.140521]...
15         [[-8.585145, 41.164857], [-8.584146, 41.164704...
16         [[-8.584263, 

In [161]:
lats =  pd.Series([np.array([point[1] for point in poly],dtype=np.float32) for poly in test_polyline])

In [233]:
split[11][2]

array([ array([ 41.1414,  41.1414,  41.1425,  41.1438,  41.1444,  41.1448,  41.1447,  41.1452,  41.1469,
        41.1482,  41.1504,  41.1512,  41.1512,  41.1522,  41.1524,  41.1525,  41.1527,  41.153 ,
        41.1545,  41.1545,  41.1545,  41.1545,  41.1545], dtype=float32),
       array([ 41.1598,  41.1599,  41.1601,  41.1605,  41.1609,  41.1615,  41.162 ,  41.1626,  41.1632,
        41.1637,  41.1642,  41.1647,  41.1651,  41.1666,  41.168 ,  41.169 ,  41.17  ,  41.1706,
        41.1707], dtype=float32),
       array([ 41.1404,  41.1404,  41.1403,  41.1404,  41.1404,  41.1406,  41.1414,  41.1428,  41.1441,
        41.1448,  41.1447,  41.1452,  41.1465,  41.1473,  41.1488,  41.1506,  41.1521,  41.1525,
        41.1527,  41.1528,  41.1528,  41.1526,  41.153 ,  41.1536,  41.154 ,  41.1545,  41.1543,
        41.1543,  41.1543,  41.1544,  41.1543,  41.1543,  41.1543,  41.1543,  41.1417,  41.1406,
        41.1543,  41.1538,  41.1538,  41.1538,  41.1532,  41.1526,  41.1524,  41.1512,  41.150

In [162]:
np.alltrue([np.allclose(lats[i],split[21][2][i]) for i in range(len(lats))])

True

Latitudes check out

In [163]:
longs =  pd.Series([np.array([point[0] for point in poly],dtype=np.float32) for poly in test_polyline])

In [241]:
split[12]

('train',
 'longitude',
 array([ array([-8.6186, -8.6185, -8.6203, -8.6222, -8.624 , -8.6267, -8.6274, -8.6302, -8.6327, -8.6317,
        -8.6299, -8.6291, -8.6291, -8.6288, -8.6287, -8.6288, -8.6308, -8.6323, -8.6311, -8.6308,
        -8.6308, -8.6308, -8.6308], dtype=float32),
        array([-8.6398, -8.6404, -8.6422, -8.6445, -8.6469, -8.65  , -8.6532, -8.6564, -8.6602, -8.6631,
        -8.6662, -8.6692, -8.6709, -8.6709, -8.6696, -8.6681, -8.6666, -8.6658, -8.6657], dtype=float32),
        array([-8.613 , -8.6134, -8.6142, -8.6148, -8.6159, -8.6166, -8.6185, -8.6206, -8.6226, -8.6251,
        -8.6274, -8.6301, -8.6319, -8.6326, -8.6313, -8.6297, -8.6288, -8.6286, -8.6287, -8.6304,
        -8.6327, -8.6351, -8.6377, -8.6404, -8.6422, -8.6441, -8.6465, -8.6486, -8.6495, -8.6498,
        -8.6498, -8.6499, -8.6499, -8.6499, -8.5994, -8.5965, -8.6501, -8.6504, -8.6504, -8.6504,
        -8.6499, -8.6494, -8.6492, -8.6497, -8.6491, -8.6491, -8.6486, -8.6484, -8.6476, -8.6459,
        -8.6

In [164]:
np.alltrue([np.allclose(longs[i],split[22][2][i]) for i in range(len(longs))])

True

Longitudes check out

In [165]:
test_data['LATITUDE'] = lats

In [166]:
test_data['LONGITUDE'] = longs

### Make Validation Set

In [227]:
data.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,LATITUDE,LONGITUDE
0,1372636858620000589,C,0,NaN,0,1372636858,0,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...","[41.1414, 41.1414, 41.1425, 41.1438, 41.1444, ...","[-8.61864, -8.6185, -8.62033, -8.62215, -8.623..."
1,1372637303620000596,B,0,7.0,1,1372637303,0,False,"[[-8.639847,41.159826],[-8.640351,41.159871],[...","[41.1598, 41.1599, 41.1601, 41.1605, 41.1609, ...","[-8.63985, -8.64035, -8.6422, -8.64445, -8.646..."
2,1372636951620000320,C,0,NaN,2,1372636951,0,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...","[41.1404, 41.1404, 41.1403, 41.1404, 41.1404, ...","[-8.61296, -8.61338, -8.61421, -8.61477, -8.61..."
3,1372636854620000520,C,0,NaN,3,1372636854,0,False,"[[-8.574678,41.151951],[-8.574705,41.151942],[...","[41.152, 41.1519, 41.1519, 41.152, 41.1519, 41...","[-8.57468, -8.57471, -8.5747, -8.57466, -8.574..."
4,1372637091620000337,C,0,NaN,4,1372637091,0,False,"[[-8.645994,41.18049],[-8.645949,41.180517],[-...","[41.1805, 41.1805, 41.18, 41.1789, 41.1785, 41...","[-8.64599, -8.64595, -8.64605, -8.6468, -8.649..."


In [273]:
def make_valid_pandas(data):
    valid = (
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        []
    )    
    for row in data.itertuples():
        trip_id = row[1]
        call_type = row[2]
        origin_call = row[3]
        origin_stand = row[4]
        taxi_id = row[5]
        time = row[6]
        day_type = row[7]
        missing_data = row[8]
        latitude = row[10]
        longitude = row[11]   
        if len(latitude) == 0:
            continue

        for ts in cuts:
            if time <= ts and time + 15 * (len(latitude) - 1) >= ts:
                # keep it
                valid[0].append(trip_id)
                valid[1].append(call_type)
                valid[2].append(origin_call)
                valid[3].append(origin_stand)
                valid[4].append(taxi_id)
                valid[5].append(time)
                valid[6].append(day_type)
                valid[7].append(missing_data)
                n = (ts - time) / 15 + 1
                valid[8].append(latitude[:n])
                valid[9].append(longitude[:n])
                valid[10].append(latitude[-1])
                valid[11].append(longitude[-1])
                valid[12].append(15 * (len(latitude)-1))
                break
    return pd.DataFrame({
        'TRIP_ID':valid[0],
        'CALL_TYPE':valid[1],
        'ORIGIN_CALL':valid[2],
        'ORIGIN_STAND':valid[3],
        'TAXI_ID':valid[4],
        'TIMESTAMP':valid[5],
        'DAY_TYPE':valid[6],
        'MISSING_DATA':valid[7],
        'LATITUDE':valid[8],
        'LONGITUDE':valid[9],
        'DESTINATION_LATITUDE':valid[10],
        'DESTINATION_LONGITUDE':valid[11],
        'TRAVEL_TIME':valid[12]
        }
    )                

In [287]:
valid_data = make_valid_pandas(data)

/home/bckenstler/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:43: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/home/bckenstler/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:44: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [288]:
np.allclose(valid_data['CALL_TYPE'],valid_split[1][2])

True

In [289]:
np.allclose(valid_data['ORIGIN_CALL'],valid_split[2][2])

True

In [290]:
np.allclose(valid_data['ORIGIN_STAND'],valid_split[3][2])

True

In [291]:
np.allclose(valid_data['TAXI_ID'],valid_split[4][2])

True

In [292]:
np.allclose(valid_data['TIMESTAMP'],valid_split[5][2])

True

In [293]:
np.allclose(valid_data['DAY_TYPE'],valid_split[6][2])

True

In [294]:
np.allclose(valid_data['MISSING_DATA'],valid_split[7][2])

True

In [295]:
np.alltrue([np.allclose(valid_data['LATITUDE'][i], valid_split[8][2][i]) for i in range(0,len(valid_data['LATITUDE']))])

True

In [297]:
np.alltrue([np.allclose(valid_data['LONGITUDE'][i], valid_split[9][2][i]) for i in range(0,len(valid_data['LATITUDE']))])

True

In [298]:
np.allclose(valid_data['DESTINATION_LATITUDE'],valid_split[10][2])

True

In [299]:
np.allclose(valid_data['DESTINATION_LONGITUDE'],valid_split[11][2])

True

In [300]:
np.allclose(valid_data['TRAVEL_TIME'],valid_split[12][2])

True

Values check out. Yay

# Clustering

## Original Data

### Mean Shift Clusters

In [310]:
from sklearn.cluster import MeanShift, estimate_bandwidth

In [303]:
split[11][1]

'latitude'

In [335]:
dests = []
for i in range(0, len(split[5][2])):
    if len(split[11][2][i]) == 0: continue
    dests.append([split[11][2][i][-1], split[12][2][i][-1]])
pts = numpy.array(dests)

In [308]:
pts.shape

(1704769, 2)

In [309]:
bw = 0.001

In [311]:
ms = MeanShift(bandwidth=bw, bin_seeding=True, min_bin_freq=5)
ms.fit(pts)
cluster_centers = ms.cluster_centers_

In [312]:
cluster_centers.shape

(3454, 2)

## Pandas

In [342]:
dests = []
for row in data[['LATITUDE','LONGITUDE']].itertuples():
    if len(row[1]) == 0: continue
    dests.append([row[1][-1], row[2][-1]])
pts2 = numpy.array(dests)

In [344]:
np.allclose(pts, pts2)

True

The points are the same. Hooray!

In [345]:
np.save(data_path+'data/cluster_centers.npy', cluster_centers)

# Feature Extraction

## Original Data

In [351]:
(1,2)+(3,)

(1, 2, 3)

In [356]:
split[8][1]

'timestamp'

In [397]:
valid_split[5][2]

array([1376502576, 1376503146, 1376502942, 1376502604, 1376502611,
       1376503017, 1376501811, 1376502255, 1376502950, 1376502635,
       1376502627, 1376502237, 1376500608, 1376502047, 1376502355,
       1376502180, 1376502802, 1376502603, 1376501182, 1376503155,
       1376502617, 1376502994, 1376502822, 1376502466, 1376503126,
       1376501063, 1376502919, 1376503023, 1376502971, 1376503012,
       1376503077, 1376500095, 1376502674, 1376503111, 1376502451,
       1376501436, 1376500679, 1376502483, 1376502952, 1376502546,
       1376503135, 1376502684, 1376502852, 1376502866, 1376502678,
       1376503196, 1376502511, 1376502716, 1376501962, 1376503045,
       1376503048, 1376503060, 1376501941, 1376502639, 1376498899,
       1380615372, 1380616084, 1380615517, 1380615888, 1380615819,
       1380616117, 1380615616, 1380615829, 1380616031, 1380614780,
       1380616088, 1380616142, 1380615884, 1380615884, 1380615546,
       1380612513, 1380616168, 1380616051, 1380615733, 1380615

In [405]:
def get_date_data(split):
    tmp = (
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        [],
        []
    )    
    for ts in split[8][2]:
        date = datetime.datetime.utcfromtimestamp(ts)
        yearweek = date.isocalendar()[1] - 1
        tmp[0].append(numpy.int8(51 if yearweek == 52 else yearweek))
        tmp[1].append(numpy.int8(date.weekday()))
        tmp[2].append(numpy.int8(date.hour * 4 + date.minute / 15))
    for ts in split[18][2]:
        date = datetime.datetime.utcfromtimestamp(ts)
        yearweek = date.isocalendar()[1] - 1
        tmp[3].append(numpy.int8(51 if yearweek == 52 else yearweek))
        tmp[4].append(numpy.int8(date.weekday()))
        tmp[5].append(numpy.int8(date.hour * 4 + date.minute / 15))
    for ts in valid_split[5][2]:
        date = datetime.datetime.utcfromtimestamp(ts)
        yearweek = date.isocalendar()[1] - 1
        tmp[6].append(numpy.int8(51 if yearweek == 52 else yearweek))
        tmp[7].append(numpy.int8(date.weekday()))
        tmp[8].append(numpy.int8(date.hour * 4 + date.minute / 15))
    return (
        ('train', 'yearweek', np.array(tmp[0])),
        ('train', 'weekday', np.array(tmp[1])),
        ('train', 'quarterhour', np.array(tmp[2])),
        ('test', 'yearweek', np.array(tmp[3])),
        ('test', 'weekday', np.array(tmp[4])),
        ('test', 'quarterhour', np.array(tmp[5])),
        ('valid', 'yearweek', np.array(tmp[6])),
        ('valid', 'weekday', np.array(tmp[7])),
        ('valid', 'quarterhour', np.array(tmp[8]))
    )

In [406]:
dates = get_date_data(split)

In [408]:
dates[3][2].shape

(320,)

In [365]:
dates[2][2].shape[0]

1710670

In [366]:
trn_size = dates[2][2].shape[0]

In [389]:
test_size = dates[4][2].shape[0]

In [409]:
val_size= dates[6][2].shape[0]

In [388]:
_size

320

In [424]:
train_gps_mean = [np.concatenate([split[11][2][i] for i in range(trn_size)]).mean(),
                np.concatenate([split[12][2][i] for i in range(trn_size)]).mean()]

In [425]:
train_gps_std = [np.concatenate([split[11][2][i] for i in range(trn_size)]).std(),
                np.concatenate([split[12][2][i] for i in range(trn_size)]).std()]

In [426]:
def at_least_k(k, v, pad_at_begin, is_longitude):
    if len(v) == 0:
        v = numpy.array([train_gps_mean[1 if is_longitude else 0]])
    if len(v) < k:
        if pad_at_begin:
            v = numpy.concatenate((numpy.full((k - len(v),), v[0]), v))
        else:
            v = numpy.concatenate((v, numpy.full((k - len(v),), v[-1])))
    return v

In [411]:
valid_split[9]

('valid',
 'longitude',
 array([ array([-8.64950371, -8.64968395, -8.65029621, -8.64983654, -8.64794731,
        -8.64522934, -8.64282608, -8.6416378 , -8.63961315, -8.63767815,
        -8.63641834, -8.63382626, -8.63256645, -8.63259315, -8.63254833,
        -8.63091946, -8.62826443, -8.62643719, -8.62646389, -8.62608624,
        -8.62596893, -8.62595081, -8.62570763, -8.62559128, -8.62558174,
        -8.62537479, -8.62343979, -8.62133408, -8.62027168, -8.62012768,
        -8.61866093, -8.61792278, -8.61756325, -8.61727524, -8.61649227,
        -8.61578083, -8.61471939, -8.61359406, -8.61299992, -8.61210918,
        -8.61203671, -8.61202812], dtype=float32),
        array([-8.64962101, -8.64962959, -8.64984608, -8.64981937], dtype=float32),
        array([-8.58569431, -8.58571243, -8.58622456, -8.58612633, -8.58614445,
        -8.58617115, -8.58624268, -8.58703518, -8.5863781 , -8.5848484 ,
        -8.58359718, -8.5822649 , -8.58025837, -8.57950211, -8.57837677,
        -8.57883644, -8

In [418]:
import theano

AttributeError: module 'theano' has no attribute 'tests'

In [437]:
def get_first_last_k(split, k):
    first_k = (
        [],
        [],
        [],
        [],
        [],
        []
    )
    last_k = (
        [],
        [],
        [],
        [],
        [],
        []
    )
    for i in range(trn_size):
        first_k[0].append(np.array(at_least_k(k, split[11][2][i], False, False)[:k]))
        first_k[1].append(np.array(at_least_k(k, split[12][2][i], False, True)[:k]))
        last_k[0].append(np.array(at_least_k(k, split[11][2][i], True, False)[-k:]))
        last_k[1].append(np.array(at_least_k(k, split[12][2][i], True, True)[-k:]))
    for i in range(test_size):
        first_k[2].append(np.array(at_least_k(k, split[21][2][i], False, False)[:k]))
        first_k[3].append(np.array(at_least_k(k, split[22][2][i], False, True)[:k]))
        last_k[2].append(np.array(at_least_k(k, split[21][2][i], True, False)[-k:]))
        last_k[3].append(np.array(at_least_k(k, split[22][2][i], True, True)[-k:]))
    for i in range(val_size):
        first_k[4].append(np.array(at_least_k(k, valid_split[8][2][i], False, False)[:k]))
        first_k[5].append(np.array(at_least_k(k, valid_split[9][2][i], False, True)[:k]))
        last_k[4].append(np.array(at_least_k(k, valid_split[8][2][i], True, False)[-k:]))
        last_k[5].append(np.array(at_least_k(k, valid_split[9][2][i], True, True)[-k:]))
    return (
        ('train', 'first_latitude', np.array(first_k[0])),
        ('train', 'first_longitude', np.array(first_k[1])),
        ('train', 'last_latitude', np.array(last_k[0])),
        ('train', 'last_longitude', np.array(last_k[1])),
        ('test', 'first_latitude', np.array(first_k[2])),
        ('test', 'first_longitude', np.array(first_k[3])),
        ('test', 'last_latitude', np.array(last_k[2])),
        ('test', 'last_longitude', np.array(last_k[3])),
        ('valid', 'first_latitude', np.array(first_k[4])),
        ('valid', 'first_longitude', np.array(first_k[5])),
        ('valid', 'last_latitude', np.array(last_k[4])),
        ('valid', 'last_longitude', np.array(last_k[5])))

In [434]:
import warnings
warnings.filterwarnings("ignore")

In [438]:
coords = get_first_last_k(split, 5)

In [440]:
coords[0][2].shape

(1710670, 5)

In [446]:
coords[4][2].shape

(320, 5)

In [447]:
coords[4]

('test',
 'first_latitude',
 array([[ 41.14852142,  41.14863968,  41.1488533 ,  41.14892578,
          41.14896393],
        [ 41.14556885,  41.14558029,  41.14576721,  41.14619064,
          41.14675903],
        [ 41.14855957,  41.1488266 ,  41.14897156,  41.14901733,  41.1485672 ],
        ..., 
        [ 41.15253448,  41.1524086 ,  41.15313721,  41.15382385,  41.1543541 ],
        [ 41.15607452,  41.15592957,  41.15547943,  41.15605545,
          41.15622711],
        [ 41.18018341,  41.17995071,  41.17948914,  41.18055344,
          41.18162537]]))

In [454]:
valid_split[8][2][0][-1]

41.145992

In [455]:
valid_split[10][2][0]

41.146263

In [500]:
np.sum([len(l) for l in data['LATITUDE']])

83409386

# Move forward with Pandas

### Prepare Train

In [ ]:
data.to_pickle(data_path+'data/train_data.pkl')

In [513]:
data['DAY_OF_WEEK'] = data['TIMESTAMP'].apply(lambda t:datetime.datetime.fromtimestamp(t).weekday())

In [515]:
data['QUARTER_HOUR'] = data['TIMESTAMP'].apply(lambda t:int((datetime.datetime.fromtimestamp(t).hour*60 + datetime.datetime.fromtimestamp(t).minute)/15))

In [517]:
data['WEEK_OF_YEAR'] = data['TIMESTAMP'].apply(lambda t:datetime.datetime.fromtimestamp(t).isocalendar()[1])

In [529]:
data['DESTINATION_LATITUDE'] = data['LATITUDE'].apply(lambda l: l[-1] if len(l) > 0 else np.nan)

In [530]:
data['DESTINATION_LONGITUDE'] = data['LONGITUDE'].apply(lambda l: l[-1] if len(l) > 0 else np.nan)

In [534]:
data = data.dropna()

In [535]:
train_gps_mean

[41.157284, -8.6161261]

In [537]:
data['LATITUDE'] = data['LATITUDE'].apply(lambda l: (l-train_gps_mean[0])/train_gps_std[0])

In [538]:
data['LONGITUDE'] = data['LONGITUDE'].apply(lambda l: (l-train_gps_mean[1])/train_gps_std[1])

In [547]:
data.columns

Index(['TRIP_ID', 'CALL_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID',
       'TIMESTAMP', 'DAY_TYPE', 'MISSING_DATA', 'POLYLINE', 'LATITUDE',
       'LONGITUDE', 'DAY_OF_WEEK', 'QUARTER_HOUR', 'WEEK_OF_YEAR',
       'DESTINATION_LATITUDE', 'DESTINATION_LONGITUDE'],
      dtype='object')

In [548]:
data['CALL_TYPE'].unique()

array([2, 1, 0])

In [518]:
def at_least_k(k, v, pad_at_begin, is_longitude):
    if len(v) == 0:
        v = numpy.array([train_gps_mean[1 if is_longitude else 0]])
    if len(v) < k:
        if pad_at_begin:
            v = numpy.concatenate((numpy.full((k - len(v),), v[0]), v))
        else:
            v = numpy.concatenate((v, numpy.full((k - len(v),), v[-1])))
    return v

In [608]:
origin_call = []
origin_stand = []
taxi_id = []
day_of_week = []
quarter_hour = []
week_of_year = []
day_type = []
first_latitude = []
first_longitude = []
last_latitude = []
last_longitude = []
destination_latitude = []
destination_longitude = []

In [609]:
k = 5

In [ ]:
def 

origin_call = []
origin_stand = []
taxi_id = []
day_of_week = []
quarter_hour = []
week_of_year = []
day_type = []
first_latitude = []
first_longitude = []
last_latitude = []
last_longitude = []
destination_latitude = []
destination_longitude = []

for i in data.index:
    latitude = data['LATITUDE'][i][:-1]
    longitude = data['LONGITUDE'][i][:-1]
    l = len(latitude)
    if l==0:
        continue
    if l < 100:
        for j in range(l):
            first_latitude.append(np.array(at_least_k(k, latitude[:j+1], False, False)[:k]))
            first_longitude.append(np.array(at_least_k(k, longitude[:j+1], False, True)[:k]))
            last_latitude.append(np.array(at_least_k(k, latitude[:j+1], False, False)[-k:]))
            last_longitude.append(np.array(at_least_k(k, longitude[:j+1], False, True)[-k:]))
            origin_call.append(data['ORIGIN_CALL'][i])
            origin_stand.append(data['ORIGIN_STAND'][i])
            taxi_id.append(data['TAXI_ID'][i])
            day_of_week.append(data['DAY_OF_WEEK'][i])
            quarter_hour.append(data['QUARTER_HOUR'][i])
            week_of_year.append(data['WEEK_OF_YEAR'][i])
            day_type.append(data['DAY_TYPE'][i])
            destination_latitude.append(data['DESTINATION_LATITUDE'][i])
            destination_longitude.append(data['DESTINATION_LONGITUDE'][i])
    else:
        indices = np.random.choice(range(l), 100, replace=False)
        for j in indices:
            first_latitude.append(np.array(at_least_k(k, latitude[:j+1], False, False)[:k]))
            first_longitude.append(np.array(at_least_k(k, longitude[:j+1], False, True)[:k]))
            last_latitude.append(np.array(at_least_k(k, latitude[:j+1], False, False)[-k:]))
            last_longitude.append(np.array(at_least_k(k, longitude[:j+1], False, True)[-k:]))
            origin_call.append(data['ORIGIN_CALL'][i])
            origin_stand.append(data['ORIGIN_STAND'][i])
            taxi_id.append(data['TAXI_ID'][i])
            day_of_week.append(data['DAY_OF_WEEK'][i])
            quarter_hour.append(data['QUARTER_HOUR'][i])
            week_of_year.append(data['WEEK_OF_YEAR'][i])
            day_type.append(data['DAY_TYPE'][i])        
            destination_latitude.append(data['DESTINATION_LATITUDE'][i])
            destination_longitude.append(data['DESTINATION_LONGITUDE'][i])            

In [ ]:
print('I"m finished')

In [ ]:
5

In [613]:
len(origin_stand)

54538180

In [584]:

data['LATITUDE'][0]

array([-0.21412922, -0.21459237, -0.19930845, -0.1817088 , -0.17414403,
       -0.16868916, -0.16982131, -0.16287406, -0.13981953, -0.12221988,
       -0.09304149, -0.08187445, -0.08151422, -0.06854605, -0.0662303 ,
       -0.06427479, -0.06211343, -0.05748194, -0.03772091, -0.03746361,
       -0.03736069, -0.03756653, -0.03772091], dtype=float32)

In [605]:
first_longitude[:22]

[array([-0.04360518, -0.04360518, -0.04360518, -0.04360518, -0.04360518]),
 array([-0.04360518, -0.04111015, -0.04111015, -0.04111015, -0.04111015]),
 array([-0.04360518, -0.04111015, -0.07276894, -0.07276894, -0.07276894]),
 array([-0.04360518, -0.04111015, -0.07276894, -0.10442773, -0.10442773]),
 array([-0.04360518, -0.04111015, -0.07276894, -0.10442773, -0.13560733], dtype=float32),
 array([-0.04360518, -0.04111015, -0.07276894, -0.10442773, -0.13560733], dtype=float32),
 array([-0.04360518, -0.04111015, -0.07276894, -0.10442773, -0.13560733], dtype=float32),
 array([-0.04360518, -0.04111015, -0.07276894, -0.10442773, -0.13560733], dtype=float32),
 array([-0.04360518, -0.04111015, -0.07276894, -0.10442773, -0.13560733], dtype=float32),
 array([-0.04360518, -0.04111015, -0.07276894, -0.10442773, -0.13560733], dtype=float32),
 array([-0.04360518, -0.04111015, -0.07276894, -0.10442773, -0.13560733], dtype=float32),
 array([-0.04360518, -0.04111015, -0.07276894, -0.10442773, -0.1356073

In [606]:
last_longitude[:22]

[array([-0.04360518, -0.04360518, -0.04360518, -0.04360518, -0.04360518]),
 array([-0.04360518, -0.04111015, -0.04111015, -0.04111015, -0.04111015]),
 array([-0.04360518, -0.04111015, -0.07276894, -0.07276894, -0.07276894]),
 array([-0.04360518, -0.04111015, -0.07276894, -0.10442773, -0.10442773]),
 array([-0.04360518, -0.04111015, -0.07276894, -0.10442773, -0.13560733], dtype=float32),
 array([-0.04111015, -0.07276894, -0.10442773, -0.13560733, -0.18286417], dtype=float32),
 array([-0.07276894, -0.10442773, -0.13560733, -0.18286417, -0.19486015], dtype=float32),
 array([-0.10442773, -0.13560733, -0.18286417, -0.19486015, -0.24429809], dtype=float32),
 array([-0.13560733, -0.18286417, -0.19486015, -0.24429809, -0.28795284], dtype=float32),
 array([-0.18286417, -0.19486015, -0.24429809, -0.28795284, -0.27048764], dtype=float32),
 array([-0.19486015, -0.24429809, -0.28795284, -0.27048764, -0.23930801], dtype=float32),
 array([-0.24429809, -0.28795284, -0.27048764, -0.23930801, -0.2249657

In [ ]:
first_latitude[:22]

In [ ]:
last_latitude[:22]

In [510]:
valid_data.to_pickle(data_path+'data/valid_data.pkl')

In [511]:
test_data.to_pickle(data_path+'data/test_data.pkl')

# MODEL

In [ ]:
n_origin_call = len(data['ORIGIN_CALL'].unique())
n_taxi_id = len(data['TAXI_ID'].unique())
n_origin_stand = len(data['ORIGIN_STAND'].unique())
n_quarter_hour = len(data['QUARTER_HOUR'].unique())
n_day_of_week = len(data['DAY_OF_WEEK'].unique())
n_week_of_year = len(data['WEEK_OF_YEAR'])
n_day_type = 3

In [ ]:
latitude_sum = lambda x: np.dot(x,cluster_centers[0])
longitude_sum = lambda x: np.dot(x, cluster_centers[1])

In [ ]:
def taxi_mlp(k, shp = cluster_centers.shape[0]):
    
    first_lat_in = Input(shape(k,))
    last_lat_in = Input(shape(k,))
    first_long_in = Input(shape(k,))
    last_long_in = Input(shape(k,))
    
    center_lats = Input(shape=(shp,))
    center_longs = Input(shape=(shp,))

    emb_names = ['origin_call', 'taxi_ID', "origin_stand", "quarter_hour", "day_of_week", "week_of_year", "day_type"]
    emb_ins = [n_origin_call + 1, n_taxi_id + 1, n_origin_stand + 1, n_quarter_hour + 1, n_day_of_week + 1, n_week_of_year + 1, n_day_type+1]
    emb_outs = [10 for i in range(0,6)]
    regs = [0 for i in range(0,6)]

    embs = [embedding_input(e[0], e[1]+1, e[2], e[3]) for e in zip(emb_names, emb_ins, emb_outs, regs)]

    x = merge([first_lat_in, last_lat_in, first_long_in, last_long_in] + [Flatten()(e[1]) for e in embs], mode='concat')

    x = Dense(500, activation='relu')(x)

    x = Dense(shp, activation='softmax')(x)

    #CHECK ON CLUSTERS!!!!
    
    y_latitude = Lambda(latitude_sum,(1,))(x)
    y_longitude = Lambda(longitude_sum,(1,))(x) 

    return Model(input = [first_lat_in, last_lat_in, first_long_in, last_long_in]+[e[0] for e in embs] + [center_longs, center_lats],
                 output = [y_latitude,y_longitude])